In [1]:
from paho.mqtt import client as mqtt_client
from datetime import datetime
import csv
import json

In [2]:
print(datetime.now())

2024-03-26 15:22:52.503499


In [4]:
broker = 'broker.emqx.io'
port = 1883
topic = "move_id/AA"
client_id = ''

message_counter = 0  

def append_to_csv(data):
    with open('not_normal_data_accelerometer.csv', mode='a', newline='') as file:
        writer = csv.writer(file)

        # Check if the file is empty and write header if it is
        if file.tell() == 0:
            writer.writerow(['date', 'x', 'y', 'z'])

        writer.writerow(data)


def connect_mqtt() -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1,client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        global message_counter  
        time = datetime.now()
        data = json.loads(msg.payload.decode())
        
        
        x = float(data['accelerometerSensor']['x'])
        y = float(data['accelerometerSensor']['y'])
        z = float(data['accelerometerSensor']['z'])
        
        
        append_to_csv([time, x, y, z])
        
        print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
        
        message_counter += 1 
        
        if message_counter >= 114:
            print("Received 114 messages, stopping the run.")
            client.disconnect()  
            client.loop_stop()    

    client.subscribe(topic)
    client.on_message = on_message


def run():
    client = connect_mqtt()
    subscribe(client)
    client.loop_forever()


if __name__ == '__main__':
    run()

C:\Users\hugod\AppData\Local\Temp\ipykernel_16988\2424661488.py:26: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1,client_id)


Connected to MQTT Broker!
Received `{"accelerometerSensor":{"x":"-0.37","y":"-9.84","z":"0.16"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"4.69","y":"-9.76","z":"-2.81"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"-1.39","y":"-10.04","z":"0.89"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"-0.66","y":"-9.91","z":"-0.93"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"-2.34","y":"-10.63","z":"-1.61"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"-9.56","y":"-10.55","z":"-3.73"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"1.59","y":"-8.96","z":"-5.2"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"-1.7","y":"-9.73","z":"0.33"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"-1.44","y":"-9.61","z":"-0.15"}}` from `move_id/AA` topic
Received `{"accelerometerSensor":{"x":"7.08","y":"-16.09","z":"-8.49"}}` from `move_id/AA` topic
Received